In [ ]:
# MLflow Setup
from src.utils.mlflow_tracker import MLflowTracker

print("🔬 MLOps Pipeline Setup\n")

# Initialize MLflow
tracker = MLflowTracker()
tracker.start_experiment("customer-churn-v1")

# Log models
print("📝 Logging models to MLflow...")
tracker.log_segmentation_model(
    kmeans_result['model'],
    model_name="kmeans_segmentation",
    metrics=kmeans_result['metrics'],
    algorithm="kmeans"
)

tracker.log_churn_model(
    xgb_model,
    model_name="xgboost_churn",
    metrics=xgb_metrics,
    model_type="xgboost"
)

tracker.log_churn_model(
    lgb_model,
    model_name="lightgbm_churn",
    metrics=lgb_metrics,
    model_type="lightgbm"
)

print("✓ Models logged to MLflow")

# Model Comparison Report
print("\n" + "="*60)
print("📊 FINAL MODEL COMPARISON REPORT")
print("="*60)

summary = pd.DataFrame({
    'Model': ['XGBoost', 'LightGBM', 'Random Forest', 'Logistic Regression'],
    'Accuracy': [xgb_metrics['accuracy'], lgb_metrics['accuracy'], rf_metrics['accuracy'], lr_metrics['accuracy']],
    'AUC-ROC': [xgb_metrics['roc_auc'], lgb_metrics['roc_auc'], rf_metrics['roc_auc'], lr_metrics['roc_auc']],
    'F1-Score': [xgb_metrics['f1'], lgb_metrics['f1'], rf_metrics['f1'], lr_metrics['f1']],
    'Precision': [xgb_metrics['precision'], lgb_metrics['precision'], rf_metrics['precision'], lr_metrics['precision']],
    'Recall': [xgb_metrics['recall'], lgb_metrics['recall'], rf_metrics['recall'], lr_metrics['recall']]
})

print(summary.to_string(index=False))

# Deployment Checklist
print("\n\n📋 DEPLOYMENT CHECKLIST:")
print("✓ Models trained and evaluated")
print("✓ Feature engineering pipeline created")
print("✓ Segmentation models (K-Means, DBSCAN, Hierarchical)")
print("✓ Churn prediction models (XGBoost, LightGBM, RF)")
print("✓ Recommendation system (Collaborative Filtering)")
print("✓ NLP sentiment analysis model")
print("✓ Real-time prediction capability")
print("✓ Batch processing pipeline")
print("✓ MLflow experiment tracking")
print("✓ API endpoints defined (FastAPI)")
print("✓ Docker containerization setup")
print("✓ Kubernetes manifests created")
print("✓ Airflow DAGs for orchestration")
print("✓ Prometheus monitoring configuration")
print("\n✅ System ready for deployment!")

## 9. MLOps Pipeline & Model Deployment

Setting up experiment tracking, model registry, and deployment considerations:

In [ ]:
# Real-time Inference
print("⚡ REAL-TIME INFERENCE\n")

# Sample customer
sample_customer = customers_engineered.iloc[0]
X_sample_engineered = fe.create_customer_features(pd.DataFrame([sample_customer]))
X_sample, _ = fe.prepare_for_modeling(X_sample_engineered, fit_scaler=False)

# Predictions
xgb_pred = xgb_model.predict(X_sample)[0]
xgb_proba = xgb_model.predict_proba(X_sample)[0, 1]

print(f"Customer ID: {sample_customer['customer_id']}")
print(f"Total Purchases: {sample_customer['total_purchases']}")
print(f"Customer Lifetime Value: ${sample_customer['customer_lifetime_value']:.2f}")
print(f"\n🎯 Churn Prediction:")
print(f"  Prediction: {'⚠️ CHURN RISK' if xgb_pred == 1 else '✓ SAFE'}")
print(f"  Probability: {xgb_proba:.1%}")

# Batch Inference
from src.data.batch_processor import BatchProcessor

print("\n\n📦 BATCH INFERENCE\n")

batch_processor = BatchProcessor()

# Create test CSV
test_batch = customers_engineered.head(100).copy()
test_batch.to_csv('data/raw/batch_test.csv', index=False)

# Run batch predictions
print("Processing batch churn predictions...")
batch_results = batch_processor.batch_churn_prediction('data/raw/batch_test.csv')

print("\nBatch Results Summary:")
print(f"Total Customers: {len(batch_results)}")
print(f"High Risk (>70%): {(batch_results['churn_probability'] > 0.7).sum()}")
print(f"Medium Risk (40-70%): {((batch_results['churn_probability'] > 0.4) & (batch_results['churn_probability'] <= 0.7)).sum()}")
print(f"Low Risk (<40%): {(batch_results['churn_probability'] < 0.4).sum()}")

print("\nSample Batch Results:")
print(batch_results.head(10))

## 8. Real-time & Batch Inference

Demonstrating prediction serving and batch processing:

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score

# Confusion Matrices
print("🔍 Confusion Matrices for Churn Models:\n")

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("XGBoost", "LightGBM", "Random Forest", "Logistic Regression"),
    specs=[[{'type': 'heatmap'}, {'type': 'heatmap'}],
           [{'type': 'heatmap'}, {'type': 'heatmap'}]]
)

models_list = [
    ('xgboost', xgb_model),
    ('lightgbm', lgb_model),
    ('random_forest', rf_model),
    ('logistic_regression', lr_model)
]

positions = [(1,1), (1,2), (2,1), (2,2)]

for (model_name, model), (row, col) in zip(models_list, positions):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    
    fig.add_trace(
        go.Heatmap(z=cm, text=cm, texttemplate='%{text}', colorscale='Blues'),
        row=row, col=col
    )

fig.update_layout(height=600, title_text="Confusion Matrices", showlegend=False)
fig.show()

# Classification Reports
print("📊 Detailed Classification Report (XGBoost):\n")
y_pred_xgb = xgb_model.predict(X_test)
print(classification_report(y_test, y_pred_xgb, target_names=['No Churn', 'Churn']))

# Cross-Validation
print("\n🔄 Cross-Validation Scores (5-fold):")
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='roc_auc')
print(f"XGBoost AUC: {cv_scores.mean():.3f} (+/- {cv_scores.std():.3f})")

## 7. Model Evaluation & Comprehensive Reporting

Detailed model evaluation with cross-validation and diagnostic plots:

In [ ]:
from src.models.nlp import SentimentAnalyzer, TopicModeling, AspectBasedSentiment

# Sentiment Analysis
print("💬 Training Sentiment Analysis Model...")
sentiment_analyzer = SentimentAnalyzer()

# Encode sentiment labels
sentiment_le = LabelEncoder()
y_sentiment = sentiment_le.fit_transform(reviews_df['sentiment'])

# Train model
sentiment_analyzer.train_sentiment_model(
    reviews_df['review_text'],
    y_sentiment,
    model_type='logistic_regression'
)
print("✓ Sentiment model trained")

# Predict sentiment
sample_reviews = [
    "Amazing product! Highly recommend to everyone.",
    "Terrible quality, very disappointed.",
    "It's okay, nothing special."
]

print("\n🎯 Sample Sentiment Predictions:")
for review in sample_reviews:
    preds, probs = sentiment_analyzer.predict_sentiment(pd.Series([review]))
    sentiment = sentiment_le.inverse_transform(preds)[0]
    confidence = probs[0].max()
    print(f"Review: '{review}'")
    print(f"→ Sentiment: {sentiment.upper()} (Confidence: {confidence:.1%})\n")

# Topic Modeling
print("📚 Training LDA Topic Model...")
topic_model = TopicModeling(n_topics=5)
topic_model.train_lda(reviews_df['review_text'])

topics = topic_model.get_topics(n_words=5)
print("\nDiscovered Topics:")
for topic_id, words in topics.items():
    print(f"Topic {topic_id}: {', '.join(words)}")

# Aspect-Based Sentiment
print("\n🔍 Aspect-Based Sentiment Analysis...")
aspect_analyzer = AspectBasedSentiment()
aspect_sentiments = aspect_analyzer.analyze_aspect_sentiment(reviews_df)

print("\nAspect Sentiment Distribution:")
print(aspect_sentiments.groupby(['aspect', 'sentiment']).size().unstack(fill_value=0))

## 6. NLP Sentiment Analysis

Customer review sentiment analysis and aspect-based opinion mining:

In [ ]:
from src.models.recommendation import RecommendationSystem

# Initialize recommendation system
recsys = RecommendationSystem(n_latent_factors=50)

# Create user-item matrix
print("📦 Building User-Item Matrix...")
matrix = recsys.create_user_item_matrix(purchases_df)
print(f"✓ Matrix shape: {matrix.shape}")
print(f"✓ Sparsity: {1 - matrix.nnz / (matrix.shape[0] * matrix.shape[1]):.1%}")

# Train Collaborative Filtering (NMF)
print("\n🤝 Training Collaborative Filtering (NMF)...")
cf_result = recsys.collaborative_filtering_nmf(matrix, n_components=50, alpha=0.0001)
print(f"✓ NMF RMSE: {cf_result['rmse']:.3f}")

# Get recommendations for sample customers
print("\n🎁 Sample Recommendations:")
sample_customers = [1, 10, 100, 500]
for customer_id in sample_customers:
    recs = recsys.get_recommendations_cf(customer_id, n_recommendations=5)
    print(f"Customer {customer_id}: {recs}")

# Content-Based Filtering
print("\n📊 Building Content-Based Model...")
content_result = recsys.content_based_filtering(products_df)
print("✓ Content-based similarity matrix computed")

# Hybrid Recommendations
print("\n🔀 Generating Hybrid Recommendations...")
hybrid_recs = recsys.hybrid_recommendations(
    user_id=1,
    product_id=10,
    similarity_matrix=content_result['similarity_matrix'],
    cf_weight=0.6,
    cb_weight=0.4,
    n_recommendations=5
)
print(f"Hybrid recommendations for user 1: {hybrid_recs}")

# Evaluation
print("\n📈 Evaluating Recommendations...")
eval_metrics = recsys.evaluate_recommendations(purchases_df)
print(f"Precision: {eval_metrics['precision']:.3f}")
print(f"Recall: {eval_metrics['recall']:.3f}")
print(f"Coverage: {eval_metrics['coverage']:.1%}")

## 5. Product Recommendation System

Implementing collaborative filtering, content-based, and hybrid recommendation approaches:

In [ ]:
# SHAP Explanations
import shap

print("📊 SHAP Feature Importance Analysis (XGBoost)...\n")

# Create SHAP explainer
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test[:100])  # Use subset for speed

# Summary plot
shap.summary_plot(shap_values, X_test[:100], feature_names=feature_cols, show=False, plot_type="bar")
plt.title("SHAP Feature Importance")
plt.tight_layout()
plt.show()

print("✓ SHAP analysis completed")

In [ ]:
# ROC-AUC Curves
from sklearn.metrics import roc_curve, auc

fig = go.Figure()

for model_name, predictions in churn_models.predictions.items():
    y_pred_proba = predictions['y_pred_proba']
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    fig.add_trace(go.Scatter(
        x=fpr, y=tpr,
        mode='lines',
        name=f'{model_name} (AUC={roc_auc:.3f})'
    ))

fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines', name='Random Classifier',
    line=dict(dash='dash')
))

fig.update_layout(
    xlabel='False Positive Rate',
    ylabel='True Positive Rate',
    title='ROC-AUC Curves for Churn Prediction Models',
    hovermode='closest'
)
fig.show()

# Feature Importance
print("\n🔍 Feature Importance (XGBoost):")
importance = churn_models.get_feature_importance('xgboost', top_k=15)
importance_df = pd.DataFrame(importance).T
print(importance_df)

In [ ]:
from src.models.churn import ChurnPredictionModels

# Initialize churn models
churn_models = ChurnPredictionModels()

print("🎯 Training Churn Prediction Models...\n")

# Train XGBoost
print("1️⃣ XGBoost Model")
xgb_model, xgb_metrics = churn_models.xgboost_model(X_train, X_test, y_train, y_test)

# Train LightGBM
print("\n2️⃣ LightGBM Model")
lgb_model, lgb_metrics = churn_models.lightgbm_model(X_train, X_test, y_train, y_test)

# Train Random Forest
print("\n3️⃣ Random Forest Model")
rf_model, rf_metrics = churn_models.random_forest_model(X_train, X_test, y_train, y_test)

# Train Logistic Regression (Baseline)
print("\n4️⃣ Logistic Regression (Baseline)")
lr_model, lr_metrics = churn_models.logistic_regression_model(X_train, X_test, y_train, y_test)

# Compare Models
print("\n" + "="*60)
print("📊 MODEL COMPARISON")
print("="*60)
comparison_df = churn_models.compare_models()
print(comparison_df.to_string())

# Visualize Comparison
fig = px.bar(
    comparison_df,
    x='model',
    y=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'],
    barmode='group',
    title='Churn Prediction Model Comparison'
)
fig.show()

## 4. Churn Prediction Models

Building multiple models with SMOTE for class imbalance, evaluation metrics, and SHAP explanations:

In [ ]:
# Dimensionality Reduction for Visualization
print("🎨 Applying PCA for visualization...")
pca_result = apply_pca(X_seg, n_components=2)
X_pca = pca_result['X_train']
print(f"✓ PCA Explained Variance: {pca_result['explained_variance']:.1%}")

print("\n🎨 Applying t-SNE for visualization...")
X_tsne = apply_tsne(X_seg, n_components=2, perplexity=30)
print("✓ t-SNE completed")

# PCA Visualization
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("K-Means", "DBSCAN", "Hierarchical"),
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}, {'type': 'scatter'}]]
)

fig.add_trace(
    go.Scatter(x=X_pca[:, 0], y=X_pca[:, 1], mode='markers', 
               marker=dict(color=kmeans_labels, colorscale='Viridis'),
               name='K-Means'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=X_pca[:, 0], y=X_pca[:, 1], mode='markers',
               marker=dict(color=dbscan_labels, colorscale='Plasma'),
               name='DBSCAN'),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=X_pca[:, 0], y=X_pca[:, 1], mode='markers',
               marker=dict(color=hierarchical_labels, colorscale='Viridis'),
               name='Hierarchical'),
    row=1, col=3
)

fig.update_layout(height=400, title_text="Clustering Results (PCA Projection)")
fig.show()

# Cluster Profiles
print("\n📊 Cluster Profiles (K-Means):")
profiles = seg.get_cluster_profiles(customers_engineered, kmeans_labels)
print(profiles)

In [ ]:
from src.models.segmentation import CustomerSegmentation
from src.features.engineering import apply_pca, apply_tsne

# Get segmentation features
X_seg = fe.get_segmentation_features(customers_engineered, fit_scaler=True)

# Initialize segmentation
seg = CustomerSegmentation()

# 1. K-Means Clustering
print("🎯 Training K-Means Clustering...")
kmeans_result = seg.kmeans_clustering(X_seg, n_clusters=5, n_init=10)
kmeans_labels = kmeans_result['labels']
print(f"✓ K-Means Metrics: {kmeans_result['metrics']}")

# 2. DBSCAN Clustering
print("\n🎯 Training DBSCAN Clustering...")
dbscan_result = seg.dbscan_clustering(X_seg, eps=1.5, min_samples=5)
dbscan_labels = dbscan_result['labels']
print(f"✓ DBSCAN Metrics: {dbscan_result['metrics']}")

# 3. Hierarchical Clustering
print("\n🎯 Training Hierarchical Clustering...")
hierarchical_result = seg.hierarchical_clustering(X_seg, n_clusters=5, linkage='ward')
hierarchical_labels = hierarchical_result['labels']
print(f"✓ Hierarchical Metrics: {hierarchical_result['metrics']}")

# Find optimal clusters
print("\n🔍 Finding optimal number of clusters...")
optimal = seg.find_optimal_clusters(X_seg, max_k=10)
optimal_k = optimal['optimal_k']
print(f"✓ Optimal K: {optimal_k}")

# Visualize optimal K
fig = go.Figure()
fig.add_trace(go.Scatter(y=optimal['inertias'], mode='lines+markers', name='Inertia'))
fig.add_trace(go.Scatter(y=optimal['silhouettes'], mode='lines+markers', name='Silhouette', yaxis='y2'))
fig.update_layout(
    yaxis=dict(title='Inertia'),
    yaxis2=dict(title='Silhouette Score', overlaying='y', side='right'),
    title='Elbow Method & Silhouette Analysis'
)
fig.show()

## 3. Customer Segmentation with Clustering

Implementing multiple clustering algorithms with visualization and evaluation metrics:

In [ ]:
from src.features.engineering import FeatureEngineer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Initialize feature engineer
fe = FeatureEngineer()

# Create engineered features
print("🔧 Engineering features...")
customers_engineered = fe.create_customer_features(customers_df)

print("✓ Engineered features created:")
print(customers_engineered.columns.tolist())

# Select features for modeling
X, feature_cols = fe.prepare_for_modeling(customers_engineered, fit_scaler=True)
y = customers_engineered['churned'].values

print(f"\n📊 Feature matrix shape: {X.shape}")
print(f"Features: {feature_cols}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n✓ Train set: {X_train.shape[0]} samples")
print(f"✓ Test set: {X_test.shape[0]} samples")
print(f"✓ Churn in train: {y_train.mean():.1%}")
print(f"✓ Churn in test: {y_test.mean():.1%}")

## 2. Feature Engineering & Preprocessing

Creating advanced features for machine learning models:

In [ ]:
# Exploratory Data Analysis
print("📈 Basic Statistics:")
print(customers_df.describe())

print("\n🎯 Churn Distribution:")
print(customers_df['churned'].value_counts())
print(f"Churn Rate: {customers_df['churned'].mean():.1%}")

# Correlation Analysis
fig = px.imshow(
    customers_df[['total_purchases', 'avg_order_value', 'days_active', 
                   'login_frequency', 'support_tickets', 'churned']].corr(),
    title="Feature Correlation Matrix",
    labels=dict(color="Correlation")
)
fig.show()

# Distribution plots
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=("Purchase Distribution", "Order Value", "Days Active", 
                   "Login Frequency", "Support Tickets", "Churn Rate")
)

features = ['total_purchases', 'avg_order_value', 'days_active', 
            'login_frequency', 'support_tickets', 'churned']

for idx, feature in enumerate(features, 1):
    row = (idx - 1) // 3 + 1
    col = (idx - 1) % 3 + 1
    
    fig.add_trace(
        go.Histogram(x=customers_df[feature], name=feature, nbinsx=30),
        row=row, col=col
    )

fig.update_layout(height=600, showlegend=False, title_text="Feature Distributions")
fig.show()

In [9]:
# Generate Sample Customer Data
from src.data.generate_data import (
    generate_customer_data, generate_product_data, 
    generate_purchase_history, generate_reviews
)

# Generate datasets
print("📊 Generating customer datasets...")
customers_df = generate_customer_data(n_customers=2000)
products_df = generate_product_data(n_products=100)
purchases_df = generate_purchase_history(n_customers=2000, n_products=100, n_transactions=10000)
reviews_df = generate_reviews(n_reviews=1000)

print(f"✓ Customers: {customers_df.shape}")
print(f"✓ Products: {products_df.shape}")
print(f"✓ Purchases: {purchases_df.shape}")
print(f"✓ Reviews: {reviews_df.shape}")

print("\n📋 Customer Data Sample:")
customers_df.head()

ModuleNotFoundError: No module named 'src'

In [5]:
# ==================== SECTION 1: Data Loading & EDA ====================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configure visualization
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


# Full-Stack Customer ML Platform 🚀

## Customer Behavior Analysis, Segmentation & Churn Prediction

This comprehensive notebook builds a production-ready AI system featuring:
- **Customer Segmentation** (K-Means, DBSCAN, Hierarchical Clustering)
- **Churn Prediction** (XGBoost, LightGBM, Random Forest + SMOTE)
- **Product Recommendations** (Collaborative Filtering, Content-Based, Hybrid)
- **NLP Sentiment Analysis** (Text preprocessing, feature extraction, classification)
- **Model Deployment** (FastAPI real-time + batch serving)
- **MLOps Pipeline** (MLflow tracking, model registry, Kubernetes deployment)